# Imports

In [2]:
import os
import json
import yaml
import torch
import random
import detectron2

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from utils.functions import grab_certain_file

# Register Dataset

In [ ]:
def get_dataset_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)
    
    dataset_dicts = []
    for idx, annots in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, annots["filename"])
        height, width = cv.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annotations = annots["regions"]
        objs = []
        for _, anno in annotations.items():
            assert not anno["region_attributes"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]
            obj = {"bbox": [np.min(px),
                            np.min(py),
                            np.max(px),
                            np.max(py)
                           ],
                   "bbox_mode": BoxMode.XYXY_ABS,
                   "segmentation": [poly],
                   "category_id": anno["category"]
                  }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
classes = ["building"]
colors = [(249, 180, 45)]

for d in ["train", "val"]:
    DatasetCatalog.register(d, lambda d=d: get_dataset_dicts(os.path.join("Spacenet", d)))
    MetadataCatalog.get(d).thing_classes = classes
    MetadataCatalog.get(d).thing_colors = colors

# Visualize Ground Truths

In [ ]:
metadata = MetadataCatalog.get("train")
dataset_dicts = get_dataset_dicts("Spacenet/train")

for d in random.sample(dataset_dicts, 3):
    img = cv.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1, instance_mode=ColorMode.IMAGE)
    out = visualizer.draw_dataset_dict(d)
    print(d["file_name"])
    plt.figure(figsize=(20, 20))
    plt.imshow(out.get_image()[:, :, ::-1])
    plt.show()

# Configurations

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")
cfg.SOLVER.BASE_LR = 0.0005
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.OUTPUT_DIR = "Spacenet/output"

# Save your configurations for multi-GPU use
with open("Spacenet/SpacenetD2cfg.yaml", "w") as file:
    yaml.dump(cfg, file)

# Train

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Inference

If you are just inferencing, make sure you define the cfg above. Run the configuration block above and the block below afterwards.

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)
metadata = MetadataCatalog.get("train")

# Overwriting configs for inference
with open("Spacenet/SpacenetD2cfg.yaml", "w") as file:
    yaml.dump(cfg, file)

## Built-in Image Inference

In [ ]:
!python inference/demo.py \
--config-file Spacenet/SpacenetD2cfg.yaml \
--input Spacenet/test/*.png \
--output Spacenet/predicitions \
--confidence-threshold 0.5

## Built-in Video Inference

In [ ]:
!python inference/demo.py \
--config-file Spacenet/SpacenetD2cfg.yaml \
--video-input example.mp4 \
--output example_output.mp4 \
--confidence-threshold 0.5

## Homemade Inference

In [ ]:
def image_pred_detectron2(files, file_path, dest, metadata, predictor):
    for item in tqdm(files, desc = "Inferencing on testing images", ncols = 150, bar_format="{l_bar}{bar:10}{r_bar}"):
        destination = os.path.join(dest, item)
        in_frame = cv.imread(os.path.join(file_path, item))
        outputs = predictor(in_frame[:, :, ::-1])
        v = Visualizer(in_frame[:, :, ::-1], metadata=metadata, instance_mode = ColorMode.SEGMENTATION)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        out_frame = v.get_image()[:, :, ::-1]
        cv.imwrite(destination, out_frame)